# Test of Neural Networks on the Elzermann readout data

In [2]:
import os, pickle

import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt


import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

# Colors using scheme
c = ["#" + i for i  in "264653-2a9d8f-e9c46a-f4a261-e76f51".split("-")]
from sklearn.metrics import roc_curve, roc_auc_score


def fft_abs_phase(input):
    transformed = np.fft.rfft(input)
    output      = np.concatenate([np.abs(transformed), np.cos(np.angle(transformed)), np.sin(np.angle(transformed))],
                                  axis = 1)
    return output
X_train, X_test, y_train, y_test = pd.read_pickle("/mnt/c/Users/johan/Desktop/CMT_project/data/train_test_data.dat")

## Imports

In [10]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from keras_lr_finder import LRFinder

import tensorflow.keras.backend as K


from sklearn.preprocessing import RobustScaler

### Common training parameters

In [83]:
epochs     = 50
batch_size = 64
timesteps  = 220
val_split  = 0.15
patience   = 3

scaler              = RobustScaler()
X_train_scaled      = scaler.fit_transform(X_train)
X_test_scaled       = scaler.transform(X_test)

## Feed-Forward

In [36]:
# Parameters
name           = "feed_forward_network"
dropout        = 0.25
hidden_states  = 64 
learning_rate  = 5e-3
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x     = Dense(hidden_states, activation = "relu")(input)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)

model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 220)]             0         
                                                                 
 dense_90 (Dense)            (None, 64)                14144     
                                                                 
 dropout_72 (Dropout)        (None, 64)                0         
                                                                 
 dense_91 (Dense)            (None, 64)                4160      
                                                                 
 dropout_73 (Dropout)        (None, 64)                0         
                                                                 
 dense_92 (Dense)            (None, 64)                4160      
                                                                 
 dropout_74 (Dropout)        (None, 64)                0  

## 1D-conv:


In [66]:
from tensorflow.keras.layers import Conv1D, Flatten

In [70]:
# Parameters
name           = "1D_conv"
dropout        = 0.5
filter_sizes   = (32,  16, 16, 8)
kernal_sizes   = (101, 51, 25, 10)
hidden_states  = 32 
learning_rate  = 1e-3
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x = input
x = tf.expand_dims(x, -1)

for fs, ks in zip(filter_sizes, kernal_sizes):
    x     = Conv1D(fs, kernel_size = ks, activation = "relu")(x)

x     = Flatten()(x)

# x     = tf.squeeze(x, -1)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_51 (InputLayer)       [(None, 220)]             0         
                                                                 
 tf.expand_dims_9 (TFOpLambd  (None, 220, 1)           0         
 a)                                                              
                                                                 
 conv1d_89 (Conv1D)          (None, 120, 32)           3264      
                                                                 
 conv1d_90 (Conv1D)          (None, 70, 16)            26128     
                                                                 
 conv1d_91 (Conv1D)          (None, 46, 16)            6416      
                                                                 
 conv1d_92 (Conv1D)          (None, 37, 8)             1288      
                                                          

## LSTM: 


In [71]:
from tensorflow.keras.layers import LSTM

In [75]:
# Parameters
name           = "LSTM"
dropout        = 0.15
hidden_states  = 32 
units          = (4, 8, 4)
learning_rate  = 1e-2
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x = input
x = tf.expand_dims(input, -1)

for u in units:
    x     = LSTM(u, dropout = dropout, return_sequences = True)(x)

x     = Flatten()(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_55 (InputLayer)       [(None, 220)]             0         
                                                                 
 tf.expand_dims_13 (TFOpLamb  (None, 220, 1)           0         
 da)                                                             
                                                                 
 lstm_9 (LSTM)               (None, 220, 4)            96        
                                                                 
 lstm_10 (LSTM)              (None, 220, 8)            416       
                                                                 
 lstm_11 (LSTM)              (None, 220, 4)            208       
                                                                 
 flatten_6 (Flatten)         (None, 880)               0         
                                                          

## TCN

In [77]:
from tcn import TCN, tcn_full_summary

In [92]:
# Parameters
name           = "TCN"
dropout        = 0.25
hidden_states  = 32
nb_filters     = 8
kernel_size    = 3
dilations      = (1, 2, 4, 8, 16, 32) 
learning_rate  = 1e-3
test_lr        = False
test_interval  = (1e-9, 1)
fit_model      = True
test_model     = True
save_preds     = True

input = Input(shape = (timesteps))

x     = input
x     = tf.expand_dims(input, -1)

x     = TCN(
    nb_filters  = nb_filters,
    kernel_size = kernel_size,
    dilations   = dilations,
    dropout_rate = dropout,
    return_sequences = True)(x)

x     = Flatten()(x)

x     = Dense(hidden_states, activation = "relu")(x)
x     = Dropout(dropout)(x)

x     = Dense(hidden_states // 4, activation = "relu")(x)
x     = Dropout(dropout)(x)

out   = Dense(1, activation = "sigmoid")(x)

model = Model(inputs = input, outputs = out)



model.compile(
    loss    = BinaryCrossentropy(from_logits = False),
    metrics   = [BinaryAccuracy(name = "Acc"), AUC(name = "AUC", from_logits = True)],
    optimizer = Adam(learning_rate = learning_rate)
)

model.summary()

if test_lr:
    finder = LRFinder(model)
    finder.find(X_train_scaled, y_train, test_interval[0], test_interval[1], batch_size = batch_size)
    finder.plot_loss()

if fit_model:
    model.fit(
        X_train_scaled, y_train,
        epochs = epochs,
        batch_size = batch_size,
        validation_split = val_split,
        callbacks = EarlyStopping("val_loss", patience = patience, restore_best_weights = True)
        )

if test_model:
    model.evaluate(X_test_scaled, y_test)

    test_preds = model(X_test_scaled)

    if save_preds:
        with open(f"../predictions/{name}.dat", "wb") as file:
            pickle.dump((test_preds, y_test), file)


Model: "model_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_68 (InputLayer)       [(None, 220)]             0         
                                                                 
 tf.expand_dims_26 (TFOpLamb  (None, 220, 1)           0         
 da)                                                             
                                                                 
 tcn_16 (TCN)                (None, 220, 8)            2248      
                                                                 
 flatten_18 (Flatten)        (None, 1760)              0         
                                                                 
 dense_193 (Dense)           (None, 32)                56352     
                                                                 
 dropout_151 (Dropout)       (None, 32)                0         
                                                          